In [10]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from urllib.parse import urlparse
import pickle

In [2]:
df=pd.read_csv("webpage data.csv")

In [3]:
def clean_url(df):
    df["Url"] = df["Url"].str.replace("/", " ")
    df["Url"] = df["Url"].str.replace("-", " ")
    df["Url"] = df["Url"].str.replace("https:", "")
    df["Url"] = df["Url"].str.replace("http:", "")
    return df

In [4]:
df = clean_url(df)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion

In [6]:
# Word and character TFIDF on URLs
vec1 = TfidfVectorizer(analyzer='char', ngram_range=(1, 5), min_df=500)
vec2 = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=400)
vec_tfidf = FeatureUnion([("char", vec1), ("word", vec2)])
vec_tfidf.fit(df['Url'])
Url_tfidf = vec_tfidf.transform(df['Url'])

In [7]:
X = Url_tfidf

TAG_DICT = {"others":1, "news": 2, "publication":3, "profile": 4,
            "conferences": 5, "forum": 6, "clinicalTrials": 7,
            "thesis": 8, "guidelines": 9}

df['target'] = df.Tag.map(TAG_DICT)
y = df["target"].values

In [11]:
pickle.dump(vec_tfidf, open('vec_tfidi_transform.pkl', 'wb'))

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [9]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(C=0.1,class_weight="balanced")
log_reg.fit(X_train, y_train)

LogisticRegression(C=0.1, class_weight='balanced')

In [12]:
filename = 'web_classification.pkl'
pickle.dump(log_reg, open(filename, 'wb'))